**Introduction**

In this notebook, I aim to demonstrate how RSA encryption can be broken through Shor's algorithm. I will first include a successful transfer of encrypted data using the RSA protocol. Next, I will simulate an eavesdropper who intercepts the encrypted message and uses a classical algorithm to decrypt it. Finally, I will simulate another eavesdropper who instead uses the quantum Shor's algorithm to decrypt the message. Through this process, I will show that Shor's algorithm offers a time complexity speed up to the alternative classical algorithm for cracking RSA encryption.

**Imports**

In [1]:
# %pip install qiskit
# %pip install qiskit_ibm_provider

In [2]:
import qiskit
from qiskit import QuantumCircuit, Aer, IBMQ
from qiskit.algorithms import Shor
from qiskit.utils import QuantumInstance
from qiskit_ibm_provider import IBMProvider
import random
import math

**RSA**

Initialization

In [3]:
# primes
prime_max = 6
primes = []
for i in range(2, prime_max):
    is_prime = True
    for j in primes:
        if j > math.sqrt(i):
            break
        if i%j == 0:
            is_prime = False
            break
    if is_prime:
        primes.append(i)

# private
p, q = random.sample(primes[1:], 2)
l = ((p - 1) * (q - 1)) // math.gcd(p - 1, q - 1)

# public
n = p * q
e = random.randint(3, l - 1)
while math.gcd(e, l) != 1:
    e = random.randint(3, l - 1)

# private
d = pow(e, -1, l)

print(n, p, q)

15 5 3


**Encryption (Alice)**

Alice, the sender of the message, will first encrypt her message using RSA

In [4]:
def hash(s: str, base: int, start: str) -> int:
    ret = 0
    for x in s:
        ret = base*ret + ord(x)-ord(start)
    return ret

def unhash(code: int, base: int, start: str) -> str:
    ret = ""
    while code > 0:
        ret = chr(code%base+ord(start)) + ret
        code //= base
    return ret

In [5]:
BASE = 3
START = 'G'

plaintext = "HI"
plaincode = hash(plaintext, BASE, START)
if plaincode >= n:
    print("Message too large")
    exit(0)
print(plaincode)

ciphercode = pow(plaincode, e, n)

5


**Decryption (Bob)**

Bob, the intended recipient of the message, will now decrypt the message

In [6]:
plaincode = pow(ciphercode, d, n)
print(plaincode)
plaintext = unhash(plaincode, BASE, START)

print(plaintext)

5
HI


**Hacking**

**Brute force O(sqrt(n))**

The following algorithm has a time complexity of O(sqrt(n)), and it will be used by the classical eavesdropper when hacking the previous encrypted message transfer

In [7]:

for i in range(2, 1+int(math.sqrt(n))):
    if n % i == 0:
        hp, hq = i, n//i
hl = ((hp - 1) * (hq - 1)) // math.gcd(hp - 1, hq - 1)
hd = pow(e, -1, hl)

In [8]:
plaincode = pow(ciphercode, hd, n)
print(plaincode)
plaintext = unhash(plaincode, BASE, START)

print(plaintext)

5
HI


**Shor's algorithm O(log(n))**

Shor's algorithm will have a time complexity of O(log(n)), hacking the previous encrypted message transfer with a lower time complexity than the classical eavesdropper

In [9]:
IBMQ.enable_account('6296569d087a150d4428571fb6266291a33c4dd0e76b421a526b60550facbe7eac5aa8310d23a51d18163a8f61290ab0401ac57a0e1d4f512c08a82f10a7d7c6')
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_qasm_simulator')
instance = QuantumInstance(backend, shots=100)
factorizer = Shor(instance)
factors = factorizer.factor(N=n, a = 2).factors[0]
print(factors)

/var/folders/zk/684rggrn76sf_fhg61wgw21r0000gn/T/ipykernel_74950/4208701671.py:1: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.enable_account('6296569d087a150d4428571fb6266291a33c4dd0e76b421a526b60550facbe7eac5aa8310d23a51d18163a8f61290ab0401ac57a0e1d4f512c08a82f10a7d7c6')
/var/folders/zk/684rggrn76sf_fhg61wgw21r0000gn/T/ipykernel_74950/4208701671.py:1: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with

[3, 5]


In [10]:
hp, hq = factors[0], factors[1]
hl = ((hp - 1) * (hq - 1)) // math.gcd(hp - 1, hq - 1)
hd = pow(e, -1, hl)

In [11]:
plaincode = pow(ciphercode, hd, n)
print(plaincode)
plaintext = unhash(plaincode, BASE, START)

print(plaintext)

5
HI


**Conclusion**

As demonstrated above, the classical eavesdropper will have a greater time complexity than the quantum eavesdropper (who uses Shor's algorithm). This means that numbers which were previously too big to be realistically factored by classical algorithms will not hold up against Shor's algorithm. This not only demonstrates the power of quantum algorithms like Shor's algorithm, but also the risk that they pose to current cybersecurity systems. Even though current quantum computers are not powerful enough to use Shor's algorithm on meaningfully large inputs, this will definitely change in the coming years. Therefore, efforts to update encryption to be quantum proof are not only necessary but urgent.

**Reflection Questions**

1. If I had more time, I would have liked to try my hand at implementing the actual Shor's algorithm quantum circuit, rather than just using Qiskit's built in Shor factorizer
2. A challenge that I faced was trying to correctly implement the RSA protocol based on the wikipedia page. If I were to do this again, I might have looked at a YouTube tutorial instead
3. The course concepts that I connected to this was correctly using Shor's algorithm by using it to factor a number into its prime factors
4. This project relates to my own interests, because I've always been interested in cybersecurity and the algorithm that might one day turn the field on its head
5. This project definitely relates the societal impact of AI on the future, as it demonstrates that the quantum algorithm threatening RSA encryption can already be implemented and used (although not powerful enough currently to break standard encryption)
6. The thing that I enjoyed most about this project was being able to actually use an algorithm we learned from class to do something that feels meaningful (breaking RSA)

**Works Cited**

“How Quantum Computer Break the Internet... Starting Now.” Performance by Derek Muller, YouTube, YouTube, 20 Mar. 2023, https://www.youtube.com/watch?v=-UrdExQW0cs. Accessed 7 Apr. 2023. 


Team, The Qiskit. “Shor's Algorithm.” Qiskit.org, Data 100 at UC Berkeley, 6 Apr. 2023, https://qiskit.org/textbook/ch-algorithms/shor.html. 